In [408]:
import pandas as pd
import requests
import time
import random
import sqlite3
import re

from pprint import pprint as pp
from bs4 import BeautifulSoup

In [409]:
def request_get_html(url):
    html = requests.get(url, auth=('user', 'pass'))
    return html.text


def format_string_result(string):
    result = ' '.join(string.split())
    return result.upper()


def calculate_wait_time(lower_limit, upper_limit):
    return random.uniform(lower_limit, upper_limit)


def get_value_from_html(html, tag_type, class_name, tag_attrib=None, str_replace=None):
    try:
        if tag_attrib is not None:
            value = html.find(tag_type, {"class": class_name})[tag_attrib]
        
        else: 
            value = html.find(tag_type, {"class": class_name}).text
    
    except:
        return "N/A"
    
    if str_replace:
        value = value.replace(str_replace, "")
    
    return format_string_result(value)

## Get the latest sold items 

### Available Categories
- Jackets & Coats
- Jeans
- Pants & Jumpsuits
- Short 
- Tops



### Available Genders/ Groups
- Women
- Men
- Kids
- Home
- Pets


The idea here is (ethically) retrieve data about sold items from the Poshmark ressellers website. 

If there are any breaches of user agreements here please get in contact with me! 

In [411]:
# URL Variables
base_url = "https://poshmark.com"
brand = 'lululemon_athletica'
gender = "Women"
category = "Tops"
query_filter = "?availability=sold_out"
page_limit = 5

In [412]:
df = pd.DataFrame(columns=['item_id', 'date_added', 'gender', 'category', 'name', 'size', 'brand', 'list_price', 'sale_price', 'condition', 'link', 'likes', 'comments'])

for i in range(page_limit):
        
    url = base_url + "/brand/" + brand + "-" + gender + "-" + category + query_filter + "&max_id=" + str(i+1)
    soup = BeautifulSoup(request_get_html(url), 'html.parser')
    

    for html_details in soup.find_all("div", {"class": "card card--small"}):
        #   Get Item Name 
        item_name = get_value_from_html(html_details, "img", "ovf--h", tag_attrib="alt")

        #   Get Item Link
        link = base_url + get_value_from_html(html_details, "a", "tile__title tc--b", tag_attrib="href").lower()
        
        #   Get the property listing ID     
        item_id = get_value_from_html(html_details, "a", "tile__title tc--b", tag_attrib="data-et-prop-listing_id")

        #   Get the date added (need to regex a cloudfront url)
        date_added = get_value_from_html(html_details, "img", "ovf--h", tag_attrib="src")
        
        if date_added == 'N/A':
            date_added = get_value_from_html(html_details, "img", "ovf--h", tag_attrib="data-src")

        date_added = re.search('\d{4}\/\d{2}\/\d{2}', date_added)
        date_added = date_added.group(0)
            
            
         #   Get List Price
        list_price = get_value_from_html(html_details, "span", "p--l--1 tc--lg td--lt", str_replace="$")  
        
        #   Get Sale Price
        sale_price = get_value_from_html(html_details, "span", "p--t--1 fw--bold", str_replace="$")        

        #   Get Condition - Not always known
        condition = get_value_from_html(html_details, "span", "condition-tag all-caps tr--uppercase condition-tag--small")

        #   Get Size - Not always known
        size = get_value_from_html(html_details, "a", "tile__details__pipe__size ellipses", str_replace="Size: ")

        #   Get Brand - Not always known
        brand = get_value_from_html(html_details, "a", "tile__details__pipe__brand ellipses")
        
        #   Get the amount of likes the picture recieved 
        likes_div = html_details.find('div', {"class": 'social-action-bar tile__social-actions'})

        try:
            likes = likes_div.find('span').text
        except:
            likes = 0
        
        #   Get the amount of comments the picture recieved 
        comments_div = html_details.find('div', {"class": 'd--fl ai--c jc--sb'})
        
        try:
            comments = comments_div.find('span').text
        except:
            comments = 0
        

        #   Add to the dataframe
        df = df.append({
            "item_id": item_id,
            "date_added": date_added,
            "gender": gender,
            "category": category,
            'name': item_name, 
            'size': size, 
            "brand": brand, 
            "list_price": list_price,
            'sale_price': sale_price, 
            'condition': condition,
            'link': link,
            'likes': likes,
            'comments': comments
        }, ignore_index=True)
       
    
#     Wait to pull the next page for a few seconds   
    time.sleep(calculate_wait_time(1, 3))


In [413]:
# for html_details in soup.find_all("div", {"class": "card card--small"}):

#     comments_div = html_details.find('div', {"class": 'd--fl ai--c jc--sb'})
# #   
#     try:
#         coments = comments_div.find('span').text
#     except:
#         coments = 0
        
#     print(coments)
df.describe()

,item_id,date_added,gender,category,name,size,brand,list_price,sale_price,condition,link,likes,comments
count,240,240,240,240,240,240,240,240,240,240,240,240,240
unique,240,93,1,1,222,18,1,39,61,3,240,32,9
top,6149E0462E8E44B9A0EF49CE,2021/10/26,Women,Tops,LULULEMON TANK,6,LULULEMON ATHLETICA,0,30,N/A,https://poshmark.com/listing/lululemon-cool-ra...,3,0
freq,1,73,240,240,6,55,240,58,22,207,1,33,192


## Create the Database

In [414]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


def select_all_tasks(conn, table_name):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM {table_name}")

    print_query_result(cur)
    
def print_query_result(cursor):
    
    rows = cursor.fetchall()

    for row in rows:
        print(row)
        

## 1. Create the Table

In [415]:
# Create the table if it does not exist
conn = create_connection('db/sold_items.db')
conn.execute('''CREATE TABLE IF NOT EXISTS sold_items_women
         (ID TEXT PRIMARY KEY     NOT NULL,
         gender            TEXT    NOT NULL,
         category          TEXT     NOT NULL,
         name              CHAR(100),
         list_price        CHAR(10),
         sale_price        CHAR(10),
         condition         CHAR(10),
         link              CHAR(100),
         likes             CHAR(4),
         comments          CHAR(4),
         date_added        CHAR(10))
         ;''')
conn.close()

## 2. Insert the data into the database

In [419]:
conn = create_connection('db/sold_items.db')

for index, row in df.iterrows():
    
    name = row["name"].replace('"', '')
    insertion_query = f'''
        INSERT INTO sold_items_women (ID,gender,category,name,list_price,sale_price,condition,link,date_added, likes, comments)
        VALUES ("{row["item_id"]}", "{gender}", "{category}", "{name}", "{row["list_price"]}", "{row["sale_price"]}", "{row["condition"]}", "{row["link"]}", "{row["date_added"]}",  "{row["likes"]}",  "{row["comments"]}")
        '''
    try:
        conn.execute(insertion_query)
    except Exception as e: 
        print(e)
        continue

# Commit the changes
conn.commit()

# Close the connection
conn.close()

UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_items_women.ID
UNIQUE constraint failed: sold_ite

In [420]:
conn = create_connection('db/sold_items.db')
data = select_all_tasks(conn, 'sold_items_women')


"""
Do something with data here 

"""


conn.close()

('6149E0462E8E44B9A0EF49CE', 'Women', 'Tops', 'LULULEMON COOL RACERBACK', '68', '25', 'N/A', 'https://poshmark.com/listing/lululemon-cool-racerback-6149e0462e8e44b9a0ef49ce', '10', '0', '2021/09/21')
('616C87DBAE766F8B7FC960AF', 'Women', 'Tops', 'LULULEMON RUN: RISE AND SHINE PULLOVER MELLOW LEMON SIZE 6', '100', '57', 'N/A', 'https://poshmark.com/listing/lululemon-run-rise-and-shine-pullover-mellow-lemon-size-6-616c87dbae766f8b7fc960af', '2', '0', '2021/10/17')
('6153B327941F17B1CD056528', 'Women', 'Tops', 'LULULEMON THINK FAST PULLOVER SIZE 10', '0', '75', 'N/A', 'https://poshmark.com/listing/lululemon-think-fast-pullover-size-10-6153b327941f17b1cd056528', '6', '0', '2021/09/28')
('6153B1DC2E8E4474F7A688DF', 'Women', 'Tops', 'LULULEMON BASE RUNNER 1/2 ZIP SIZE 10', '0', '75', 'N/A', 'https://poshmark.com/listing/lululemon-base-runner-12-zip-size-10-6153b1dc2e8e4474f7a688df', '6', '0', '2021/09/28')
('611AA5943BAD6D94E1DE5CA0', 'Women', 'Tops', 'LULULEMON ORANGE RIGHT ROUND TANK SIZE 

In [421]:
conn = create_connection('db/sold_items.db')
cur = conn.cursor()
cur.execute('''Select count(1) from sold_items_women where cast(sale_price as int)''')
print_query_result(cur)

(240,)


- User inputs
    - Brand
    - Size 
    - Type

 
- This is going to be used to estimate the resale price

What we need to do is take this input, retrieve the data from the database, and calculate average sale value for 
    - This particular size
    - Other sizes
    - What they normally list for versus what they sell for
    

In [423]:
df.describe()

,item_id,date_added,gender,category,name,size,brand,list_price,sale_price,condition,link,likes,comments
count,240,240,240,240,240,240,240,240,240,240,240,240,240
unique,240,93,1,1,222,18,1,39,61,3,240,32,9
top,6149E0462E8E44B9A0EF49CE,2021/10/26,Women,Tops,LULULEMON TANK,6,LULULEMON ATHLETICA,0,30,N/A,https://poshmark.com/listing/lululemon-cool-ra...,3,0
freq,1,73,240,240,6,55,240,58,22,207,1,33,192
